# Example 2: Query multiple items using paging
See more about paging here: https://graphql.org/learn/pagination/

In [ ]:
from srgssr_publication_data_api import PublicationDataApi
import os

## 1. Check the API endpoint and initialize client

In [ ]:
%env PDP_API

In [ ]:
client = PublicationDataApi(os.environ['PDP_API'], os.environ['USER_NAME'], os.environ['USER_PASSWORD'])

## 2. Show available queries

In [ ]:
# first=[number of queries to retrieve, e.g. *first* 10 queries]
# after=[id or cursor position from which to start]
client.schema.Query

## 3. Configure the query operation with paging
To query pages programmatically we parametrize the query

In [ ]:
from sgqlc.types import Variable, non_null

op = client.query_op(name='faroItems', variables={'first':non_null(int), 'after':str})

In [ ]:
# return current query (empty)
op

In [ ]:
# select query type and configure with argument
faro = op.faro_items(first=Variable('first'), after=Variable('after'))

In [ ]:
# return current query (configured)
op

## 4. Select fields of interest

In [ ]:
# select fields to be retured
faro.edges.id()
faro.edges.program.date()
faro.edges.program.title()
faro.edges.producer()
faro.cursor()

In [ ]:
# return current query
op

## 5. Retrieve the data

In [ ]:
# constants
page_size = 100
page_requests = 100

In [ ]:
# initialize with first query
data = client.run_query(op, {'first':page_size})

results = data.faro_items

if not results.edges:
    print('No items!', file=sys.stderr)
    raise SystemExit()

In [ ]:
from tqdm.notebook import tnrange
for i in tnrange(page_requests, desc="Downloading data"):
    # query page
    data = client.run_query(op, {'first':page_size, 'after':results.cursor})
    
    # convert page to object oriented results
    page = data.faro_items

    # extract new edges and update cursor
    results.edges += page.edges
    results.cursor = page.cursor

In [ ]:
print(f'INFO - Retrieved {len(results.edges):,} FaroItems')

## 6. Plot the year distribution of first display

In [ ]:
from collections import Counter
from datetime import date

# count year occurrences
c = Counter(date.fromisoformat(x.program.date).year for x in results.edges if x.program.date)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)

In [ ]:
with plt.style.context('fivethirtyeight'):
    plt.bar(c.keys(), c.values())

## 7. Load data into a `pandas` dataframe

In [ ]:
import pandas as pd

In [ ]:
results.edges[0].__fields_cache__

In [ ]:
# todo: find better way to import data into pandas
df = pd.DataFrame([x.__json_data__ for x in results.edges])

In [ ]:
df

In [ ]:
(df
 .groupby('producer')
 .filter(lambda x: len(x) > 100)
 .producer
 .value_counts()
 .plot
 .pie(fontsize=30));